# HW06 Collocation Embeddings
### 1. Convert bnc data -> required format

##### Make verb_noun collocation tokens
* Keep the noun 
* He is arrested for his crime => He is arrest_crime for his crime


In [82]:
import pprint
import itertools

In [48]:
sentences = [ line.strip() for line in open('bnc_tagged.h') ]

In [112]:
testSentences = [sentences[i] for i in range(10000)]

In [87]:
VBs = {'VB','VBG','VBN','VBP','VBZ'} #VBN 被動 #'VBD'be 動詞
NNs = {'NN','NNS','NNP','NNPS'}
Preps = {'with','in','on','at','of','for','to','about'}

In [59]:
def writeFile(sent,fileName):
    fo = open(fileName, "a")
    fo.write(sent+'\n')

In [91]:
len(sentences)

4880632

In [123]:
%%time
for item in testSentences:
    infos = item.split('\t')
    standSentence = infos[1].split()
    tags = infos[2].split()
    
    V_index = ''
    
    for i,tag in enumerate(tags):
        if tag in VBs: 
            V_index = i
        if tag in NNs and V_index != '' and abs(V_index-i)<3 :
            newVB = standSentence[V_index] + '_' + standSentence[i]
            text = ' '.join(standSentence[:V_index]) +' ' + newVB + ' '+ ' '.join(standSentence[V_index+1:])
            writeFile(text,'sents_NN.txt')
            V_index = 0
        if standSentence[i] in Preps and V_index != '' and abs(V_index-i)<3 :
            newVB = standSentence[V_index] + '_' + standSentence[i]
            text = ' '.join(standSentence[:V_index]) +' ' + newVB + ' '+ ' '.join(standSentence[V_index+1:])
            writeFile(text,'sents_P.txt')
            V_index = 0
        

Wall time: 2.67 s


##### Make verb_prep collocation tokens
* Delete the prep
* He is arrested for his crime => He is arrest_for_N his crime

### 2. Training

In [120]:
# -*- coding: utf-8 -*-

from gensim.models import word2vec

def main():
    sentences = word2vec.Text8Corpus("sents_P.txt")
    model = word2vec.Word2Vec(sentences, size=300, window=5, min_count=10, workers=4)

    # Save our model: bnc.clause.300.bin / bnc.delete.clause.300.bin
    model.wv.save_word2vec_format('bnc.delete.clause.300.bin', binary=True)


if __name__ == "__main__":
    main()

### 3. Find similarity

In [116]:
from gensim.models import KeyedVectors
import re

In [117]:
verb_noun = KeyedVectors.load_word2vec_format("bnc.clause.300.bin", binary=True)
verb_noun_clauses = [x for x in verb_noun.vocab if re.match(r'[a-z]*\_[a-z]*', x) ]

In [118]:
verb_noun.most_similar('solve_problem')

[('solve', 0.8602089881896973),
 ('resolve_problem', 0.8568793535232544),
 ('overcome_problem', 0.8410978317260742),
 ('avoid_problem', 0.8370150923728943),
 ('tackle_problem', 0.8350973725318909),
 ('address_problem', 0.8343183994293213),
 ('alleviate_problem', 0.7957615256309509),
 ('see_problem', 0.7869168519973755),
 ('understand_problem', 0.774480402469635),
 ('handle_problem', 0.7679135799407959)]

In [121]:
verb_prep = KeyedVectors.load_word2vec_format("bnc.delete.clause.300.bin", binary=True)
verb_prep_clauses = [x for x in verb_prep.vocab if re.match(r'[a-z]*\_[a-z]*', x) ]

In [122]:
verb_prep.most_similar('arrest_for')

[('imprison_for', 0.824950098991394),
 ('prosecute_for', 0.823798418045044),
 ('sentence_for', 0.7677362561225891),
 ('sack_for', 0.7643067240715027),
 ('detain_for', 0.7507451772689819),
 ('punish_for', 0.7422752976417542),
 ('fin_for', 0.7385950088500977),
 ('dismiss_for', 0.7209904193878174),
 ('jail_for', 0.715844452381134),
 ('expel_for', 0.7147839069366455)]

### BONUS
* Combine both rule? V:prep:N